From discussion:
- Data collection
    * building trainging sets
- smali code -> graphs
    * extract nodes / edges
    * feed into networkx
    * build matrix
- build a table of features from graph

In [10]:
import pandas as pd
import numpy as np
import glob
import re
import networkx as nx
from collections import defaultdict
import itertools
import matplotlib.pyplot as plt

In [19]:
%load_ext autoreload
%autoreload 2
import collection as c

In [23]:
# get all filenames with .smali using glob recursive=True
smali = c.get_smali('./smali')
test = smali[:100]

In [ ]:
def create_i(line):
    if('invoke-' in line and 'method' not in line):
        api_call = re.search('L.+', line).group(0)[1:]
        calls.append(api_call)
#             print('api call: ', api_call)
        invoke = re.search('-\w{5,9}', line).group(0)[1:]
#             package = re.search('L.+-', line).group(0)[1:-2]
        invokeD[invoke].append(api_call)
#         except:
#             print('line', line)
#             print(re.search('L.+', line).group(0)[1:])
#             print('invoke: ' ,re.search('-\w{5,7}', line).group(0)[1:])
#             print(re.search('L.+-', line).group(0)[1:-2])
            
    else: return

In [ ]:
I = nx.Graph()
calls = list()
invokeD = {'direct': [], 'virtual':[], 'static':[], 'super':[],'interface':[]}
packageD = defaultdict(list)

for file in smali:
    with open(file) as f:
        for line in f:
            create_i(line)
        
# #             print('invoke: ', invoke)
#             if(package):
# #                 package = package.group(0)[1:-3]
# #                 print('package: ', package)
#                 method = re.search('>.+\(', line).group(0)[1:-1]
#                 packageD[package].append(method)
# #                 print('method: ', method)

In [ ]:
I.add_nodes_from(calls)
I.add_edges_from([i for i in itertools.combinations(invokeD['direct'], 2)])
I.add_edges_from([i for i in itertools.combinations(invokeD['virtual'], 2)])
I.add_edges_from([i for i in itertools.combinations(invokeD['static'], 2)])
# pos=nx.spring_layout(I)
nx.draw(I, node_size=60)

In [ ]:
def locate_method_blocks(text):
    count = 0
    append = False
    for line in text:
        if('.method' in line): append = True
        if('.end method' in line): 
            blocks[count].append(line)
            append = False
            count += 1
        if(append):
            blocks[count].append(line)
            
def find_api_calls(iterable, var):
    for i in iterable:
        if('invoke-' in i and 'method' not in i):
            api_call = re.search('L.+', i).group(0)[1:]
            var.append(api_call)

def create_edges(dic, graph):
    for key in dic:
        edges = [i for i in itertools.combinations(dic[key], 2)]
        graph.add_edges_from(edges)

In [ ]:
calls = list()
B = nx.Graph()

for f in smali:
    blocks = defaultdict(list)
    same_block = defaultdict(list)
    with open(f) as file:
        locate_method_blocks(file)
        file.seek(0)
        find_api_calls(file, calls)
        for b in blocks:
            apis = []
            find_api_calls(blocks[b], apis)
            same_block[b] = apis
        create_edges(same_block, B)


In [ ]:
B.add_nodes_from(calls)

In [ ]:
nx.adjacency_matrix(B)

In [ ]:
plt.figure(figsize=(30,50)) 
nx.draw(B, node_size=50)